In [1]:
from surprise import Dataset
from surprise import Reader
import pandas as pd
import numpy as np
import random
import nltk
import re
from unidecode import unidecode

In [2]:
pd.options.display.max_rows = 5000

# Spark initialization

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, avg

from pyspark.serializers import MarshalSerializer
from pyspark.context import SparkContext
sc = SparkContext("local", "serialization app", serializer = MarshalSerializer())
spark = SparkSession(sc)
spark.sparkContext is sc

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/19 18:49:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


True

In [3]:
p = 1
df = pd.read_csv('./data/spotify_dataset.csv', on_bad_lines='skip')
df.dropna(inplace=True)

In [4]:
df.columns = df.columns.str.replace('"', '')
df.columns = df.columns.str.replace('name', '')
df.columns = df.columns.str.replace(' ', '')

In [5]:
def clean_artist(text):
    text = (str(text)).lower()
    tok = nltk.RegexpTokenizer(r"\w+")
    word = " ".join(tok.tokenize(text))
    text = unidecode(word)
    FEAT_PAT = re.compile(r"[\s\S]+[\s]+(feat\.|ft\.|featuring|ft|feat)[\s]+[\s\S]+")
    AMP_PAT = re.compile(r"[\s\S]*(&|and|\+)[\s\S]*")
    #check if we have featured artist
    if FEAT_PAT.match(text):
        text = re.split(r"feat\.|ft\.|featuring|ft|feat", text)[0]
    
    #Remove & from all artists
    if AMP_PAT.match(text):
        text = re.split(r"&", text)[0]
    w="".join(text.split(" "))
    text=w
    return text.strip()

In [7]:
df['clean_artist'] = df['artist'].apply(clean_artist)

In [24]:
df.to_csv('./data/cleaned_df.csv', index=False)

In [4]:
spark_df = spark.read.csv("./data/cleaned_df.csv", header=True)

# Get frequencies of artists by user

In [5]:
spark_df_agg = spark_df.groupBy('user_id', 'clean_artist').agg(count('*'))

In [62]:
spark_df_agg=spark_df_agg.withColumnRenamed("count(1)","freq")

In [63]:
spark_df_agg.show(2)

+--------------------+------------+----+
|             user_id|clean_artist|freq|
+--------------------+------------+----+
|944c80d26922ae634...|  jackpenate|   2|
|944c80d26922ae634...|    froufrou|   1|
+--------------------+------------+----+
only showing top 2 rows



In [34]:
clean_df = clean_df.astype(str)

In [38]:
clean_df.to_csv('./data/hugo_df.csv', index=False)

In [39]:
spark_clean_df = spark.read.csv('./data/hugo_df.csv', header=True)

In [43]:
spark_clean_df.show(2)

+--------------------+---------+--------------------+
|             user_id|artist_id|        clean_artist|
+--------------------+---------+--------------------+
|9cc0cfd4d7d788510...|        0|       elviscostello|
|9cc0cfd4d7d788510...|        1|elviscostellothea...|
+--------------------+---------+--------------------+
only showing top 2 rows



In [64]:
combined_df = spark_df_agg.join(spark_clean_df, 
                                (spark_df_agg.clean_artist==spark_clean_df.clean_artist) & \
                                 (spark_df_agg.user_id==spark_clean_df.user_id),
                                'inner').select(spark_df_agg.user_id, spark_df_agg.clean_artist, spark_df_agg.freq, spark_clean_df.artist_id)

In [57]:
spark_df_agg.createOrReplaceTempView("s_agg")
spark_clean_df.createOrReplaceTempView("s_clean")

In [58]:
join_df = spark.sql("select * from s_agg s1, s_clean s2 where s1.clean_artist==s2.clean_artist;")

In [65]:
combined_df.count()

12855354

In [93]:
combined_df.show(2)

java.lang.NullPointerException


# 1).given user_id, return all of its songs

In [90]:
def get_all_songs(view, user_id):
    res = list(spark_df_agg.filter(spark_df_agg['user_id'] == user_id).select('clean_artist').toPandas()['clean_artist'])
#     mvv_list = list(
#         tableA.select('clean_artist').toPandas()['clean_artist']
#     )
#     res = list(spark.sql(f"SELECT clean_artist from tableA where user_id='{user_id}';").toPandas())
     
    return res

In [91]:
# res_list = [i.clean_artist for i in get_all_songs('00055176fea33f6e027cd3302289378b')]
res_list = get_all_songs('00055176fea33f6e027cd3302289378b')

res_list

Py4JError: functions does not exist in the JVM -- py4j.Py4JException: Method equalTo([class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:748)


# 2).given artist_id, return the corresponding unique artist

In [49]:
spark_clean_df.count()

12856838

In [50]:
spark_df_agg.count()

3256666

# Get all artists for user 00055176fea33f6e027cd3302289378b

In [85]:
spark_df_agg.createOrReplaceTempView("tableA")

In [7]:
spark.sql("SELECT clean_artist from tableA where user_id='00055176fea33f6e027cd3302289378b'").show()

+----------------+
|    clean_artist|
+----------------+
|           auryn|
|           bmike|
|    thecatempire|
|        thevamps|
|        ollymurs|
|       littlemix|
|    austinmahone|
|        blink182|
|       beamiller|
|      simpleplan|
|  christinaperri|
|     onerepublic|
|5secondsofsummer|
|      jakemiller|
|   meghantrainor|
|    jannikbrunke|
|         thefray|
|          beckyg|
|       cimorelli|
|  abigailbreslin|
+----------------+
only showing top 20 rows



# Create a lookup table

In [4]:
sc.stop()

In [48]:
size = lambda x: len(x)
df_freq = df.groupby(['user_id', 'artist']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist', 'freq']].sort_values(['freq'], ascending=False)
df_freq.head()

,user_id,artist,freq
2870626,defced0ece4ce946160b0d2698142eac,Vitamin String Quartet,3346
502897,26b51e580277e131f87e4c7ee4c0887a,Vitamin String Quartet,3306
844217,414050deadb38aafd8d4ad22ca634055,Vitamin String Quartet,2587
2798380,d993ff8f2de226e2c6803e47a22e9d7e,Lata Mangeshkar,2281
22579,014e695cc6df96011b90a5beb3206012,Ilaiyaraaja,2242


In [63]:
a = df_freq.artist.unique().tolist()
print(len(a))

289821


In [83]:
import re
remove_ft = r"[\s\S]+[\s]+(feat\.|ft\.|featuring|ft|feat)[\s]+[\s\S]+"
r = re.compile(remove_ft, re.IGNORECASE)

filtered_list = list(filter(r.match, a))

print(len(filtered_list))

13717


# Checks df for artist. Get's top 5 songs for each artist

In [55]:
def get_top5_songs(artist):
    
    return list(df[df['artist'] == artist]['track'].value_counts()[0:5].index)
#     songs = df.loc[df.arist]

In [56]:
get_top5_songs('David Cochran Heath')

['Psalm 63', 'Psalm 4', 'Psalm 114', 'Psalm 126', 'Job 2']

key:'kanye west'
value: ['gold digger', 'stronger', 'sdafasdf','fasdasdf', 'i miss kim'] *list should be sorted in decr order of frequency of song in database


In [23]:
clean_df = pd.read_csv('./data/cleaned_df.csv')

In [25]:
artist= pd.unique(clean_df['clean_artist'].values.ravel())
artist = pd.Series(np.arange(len(artist)), artist)
clean_df["artist_id"] = clean_df[['clean_artist']].applymap(artist.get)

In [28]:
clean_df = clean_df[['user_id','artist_id', 'clean_artist']]

In [11]:
artist_track_df = pd.DataFrame(df_freq["artist_id"].unique(), columns = ["artist_id"])

In [12]:
artist_track_df["artist"] = df_freq["artist"].unique()
df_freq.drop('artist', axis=1, inplace=True)
df_freq.head()

,user_id,freq,artist_id
0,00055176fea33f6e027cd3302289378b,10,713
1,00055176fea33f6e027cd3302289378b,1,132909
2,00055176fea33f6e027cd3302289378b,3,7994
3,00055176fea33f6e027cd3302289378b,8,698
4,00055176fea33f6e027cd3302289378b,1,8100


In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(1,5))

In [14]:
df_freq[["freq"]] = scaler.fit_transform(df_freq[["freq"]])
df_freq.head()

,user_id,freq,artist_id
0,00055176fea33f6e027cd3302289378b,1.010762,713
1,00055176fea33f6e027cd3302289378b,1.000000,132909
2,00055176fea33f6e027cd3302289378b,1.002392,7994
3,00055176fea33f6e027cd3302289378b,1.008371,698
4,00055176fea33f6e027cd3302289378b,1.000000,8100


In [15]:
reader = Reader(rating_scale=(1, 5))

In [16]:
data = Dataset.load_from_df(df_freq[['user_id', 'artist_id', 'freq']], reader)

In [17]:
from surprise.model_selection import cross_validate
from surprise import NormalPredictor

In [18]:
# from surprise import SVD
# from surprise.model_selection import KFold
# from surprise import accuracy
# # define a cross-validation iterator
# kf = KFold(n_splits=5)
# algo = SVD()
# for trainset, testset in kf.split(data):
#     # train and test algorithm.
#     algo.fit(trainset)
#     predictions = algo.test(testset)
#     # Compute and print Root Mean Squared Error
#     accuracy.rmse(predictions, verbose=True)

# Let's try to get predictions from SVD

In [19]:
from surprise import SVD
from surprise.model_selection import cross_validate



In [21]:
svd = SVD(verbose=True, n_epochs=50)

In [23]:
cross_validate(svd, data, measures=['RMSE'], cv=2, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing

{'test_rmse': array([0.02259635, 0.02275703]),
 'fit_time': (287.2614200115204, 282.93003702163696),
 'test_time': (21.067522048950195, 20.038546085357666)}

In [24]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49


In [26]:
preds = list()
for id in df_freq.artist_id.values:
    preds.append(svd.predict(uid="00055176fea33f6e027cd3302289378b", iid=id))


    

In [ ]:
# x = sorted(df_freq.freq.unique())
# x

In [28]:
preds = sorted(preds, key=lambda x: x.est, reverse=True)

In [31]:
preds[:20]

[Prediction(uid='00055176fea33f6e027cd3302289378b', iid=101477, r_ui=None, est=1.5538967097253304, details={'was_impossible': False}),
 Prediction(uid='00055176fea33f6e027cd3302289378b', iid=101477, r_ui=None, est=1.5538967097253304, details={'was_impossible': False}),
 Prediction(uid='00055176fea33f6e027cd3302289378b', iid=101477, r_ui=None, est=1.5538967097253304, details={'was_impossible': False}),
 Prediction(uid='00055176fea33f6e027cd3302289378b', iid=101477, r_ui=None, est=1.5538967097253304, details={'was_impossible': False}),
 Prediction(uid='00055176fea33f6e027cd3302289378b', iid=101477, r_ui=None, est=1.5538967097253304, details={'was_impossible': False}),
 Prediction(uid='00055176fea33f6e027cd3302289378b', iid=101477, r_ui=None, est=1.5538967097253304, details={'was_impossible': False}),
 Prediction(uid='00055176fea33f6e027cd3302289378b', iid=178381, r_ui=None, est=1.3791580882855914, details={'was_impossible': False}),
 Prediction(uid='00055176fea33f6e027cd3302289378b', iid

In [33]:
#unique preds
unique_preds = list()
iid=[]
for pred in preds:
    if pred.iid not in iid:
        iid.append(pred.iid)
    
    

KeyboardInterrupt: 

In [34]:
iid=[]
for pred in preds:
    iid.append(pred.iid)

In [47]:
iid=list(dict.fromkeys(iid))
iid

[101477,
 178381,
 223686,
 121377,
 84281,
 244635,
 43362,
 123507,
 224889,
 261475,
 266738,
 257963,
 68500,
 86231,
 116406,
 4464,
 149368,
 262771,
 233305,
 220779,
 81422,
 61778,
 272520,
 224165,
 62078,
 268459,
 278478,
 250606,
 48657,
 155322,
 122788,
 283426,
 82308,
 191461,
 79925,
 244073,
 250219,
 163138,
 180701,
 92246,
 120921,
 81851,
 263967,
 103438,
 187726,
 53895,
 16345,
 53726,
 113000,
 232530,
 251306,
 214627,
 150341,
 30428,
 107038,
 155492,
 230676,
 82318,
 222471,
 183101,
 190873,
 212105,
 243382,
 212727,
 9674,
 152522,
 116774,
 232548,
 256318,
 248524,
 18044,
 217074,
 261198,
 146576,
 168880,
 107784,
 237683,
 140525,
 60857,
 250525,
 103386,
 267221,
 151915,
 135931,
 230085,
 263283,
 213624,
 157307,
 289153,
 220197,
 101785,
 171093,
 102066,
 220229,
 163388,
 32868,
 8281,
 128006,
 7049,
 69260,
 59196,
 228122,
 216184,
 172940,
 128867,
 184892,
 269440,
 200830,
 131616,
 140437,
 84910,
 96021,
 273284,
 167891,
 22434

In [46]:
for i in iid[:5]:
    print(artist_track_df.loc[artist_track_df.artist_id==i].artist)

61358    Jamey Aebersold Play-A-Long
Name: artist, dtype: object
187737    David Cochran Heath
Name: artist, dtype: object
150305    Kurt Prestel
Name: artist, dtype: object
129571    Conlon Nancarrow
Name: artist, dtype: object
258621    Clipse ft. AB-Liva & Rosco P Goldchain
Name: artist, dtype: object


# Training with surprise

In [ ]:
def get_testset_accuracy(testset):
    total_matches = 0
    for item in testset:
        uid = item[0]
        preds = []
        for id in df_freq.artist_id.values:
            preds.append(svd.predict(uid=uid, iid=id))
        iid=[]
        for pred in preds:
            iid.append(pred.iid)
        iid=list(dict.fromkeys(iid))
        predicted_artists = set()
        for i in iid[:30]:
            artist = artist_df.loc[artist_df.artist_id == i].artist.values[0]
            predicted_artists.add(artist)
        known_artists = list(set(df.loc[df["user_id"] == uid].artist.values))
        #print(len(known_artists))
        total_matches += len(predicted_artists.intersection(known_artists))
    print(total_matches / (30 * len(testset)))      